##### IMPORTANT!!!!! DO NOT SCRAPE IT AGAIN UNLESS THE DATA ARE WRONG

# Web scrapping of Premier League players' statistics from all season

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq 
import re

# Page to scrap from
my_url = 'https://fbref.com/en/comps/9/history/Premier-League-Seasons'

# Opening up connection, grabbing the page
html = uReq(my_url)
bs = soup(html.read(), 'html.parser')

In [2]:
# Get an url for each PL season details
seasons_links = []
for x in bs.find_all('a', href=re.compile('^\/en/comps/9/[0-9|P]+')):
    pl = bs.find_all('a', text="Premier League")
    fa_pl = bs.find_all('a', text="FA Premier League")
    if x not in pl and x not in fa_pl:
        #print(x.attrs['href'])
        link = 'https://fbref.com' + x.attrs['href']
        print(link)
        seasons_links.append(link)

https://fbref.com/en/comps/9/Premier-League-Stats
https://fbref.com/en/comps/9/10728/2020-2021-Premier-League-Stats
https://fbref.com/en/comps/9/3232/2019-2020-Premier-League-Stats
https://fbref.com/en/comps/9/1889/2018-2019-Premier-League-Stats
https://fbref.com/en/comps/9/1631/2017-2018-Premier-League-Stats
https://fbref.com/en/comps/9/1526/2016-2017-Premier-League-Stats
https://fbref.com/en/comps/9/1467/2015-2016-Premier-League-Stats
https://fbref.com/en/comps/9/733/2014-2015-Premier-League-Stats
https://fbref.com/en/comps/9/669/2013-2014-Premier-League-Stats
https://fbref.com/en/comps/9/602/2012-2013-Premier-League-Stats
https://fbref.com/en/comps/9/534/2011-2012-Premier-League-Stats
https://fbref.com/en/comps/9/467/2010-2011-Premier-League-Stats
https://fbref.com/en/comps/9/400/2009-2010-Premier-League-Stats
https://fbref.com/en/comps/9/338/2008-2009-Premier-League-Stats
https://fbref.com/en/comps/9/282/2007-2008-Premier-League-Stats
https://fbref.com/en/comps/9/229/2006-2007-Prem

In [3]:
# For each season url, get an url to fixtures of that season
fixtures_links = []
for link in seasons_links:
    season = uReq(link)
    bs_season = soup(season.read(), 'html.parser')
    fixture = bs_season.find('a', href=re.compile('Premier-League-Scores-and-Fixtures+'))
    fixture_link = 'https://fbref.com' + fixture.attrs['href']
    fixtures_links.append(fixture_link)

In [4]:
link = fixtures_links[0]
fix = uReq(link)
bs_fix = soup(fix.read(), 'html.parser')
div = bs_fix.find('div', {'id':'all_sched'})
header_row = div.find_all('th', {'class': [' ', 'poptip','sort_default_asc', 'center']})[0:14]

In [5]:
# (continued) Get the header from table in fixtures
header = []
for h in header_row:
    print(h.attrs['data-stat'])
    header.append(h.attrs['data-stat'])

gameweek
dayofweek
date
time
squad_a
xg_a
score
xg_b
squad_b
attendance
venue
referee
match_report
notes


In [6]:
# Just removing useless cols
header.remove('xg_a')
header.remove('xg_b')

In [7]:
# Get a table of fixtures for each season
tables = []
for link in fixtures_links:
    fix = uReq(link)
    bs_fix = soup(fix.read(), 'html.parser')
    div = bs_fix.find('div', {'id':'all_sched'})
    tables.append(div)

In [8]:
# Create a dictionary with data for each statistic (column) by using for loop
data = dict()
for h in header:
    data[h] = []
    if h == 'gameweek':
        for table in tables:
            key = table.find_all('th', {'data-stat': h})
            for k in key:
                if k.get_text() == 'Wk':
                    continue
                elif len(k)==0:
                    data[h].append(0)
                else:
                    data[h].append(k.get_text())
    elif h == 'match_report':
        for table in tables:
            key = table.find_all('td', {'data-stat': h})
            for k in key:
                try:
                    a = k.find('a')
                    link = 'https://fbref.com' + a.attrs['href']
                    data[h].append(link)
                except AttributeError:
                    data[h].append(0)
    else: 
        for table in tables:
            key = table.find_all('td', {'data-stat': h})         
            for k in key:
                if len(k)==0:
                    data[h].append(0)
                else:
                    data[h].append(k.get_text())

for k, v in data.items():
    print(f"{k}: {len(v)}")

gameweek: 13159
dayofweek: 13159
date: 13159
time: 13159
squad_a: 13159
score: 13159
squad_b: 13159
attendance: 13159
venue: 13159
referee: 13159
match_report: 13159
notes: 13159


In [9]:
import pandas as pd
 
pl = pd.DataFrame(data=data)
pl.head(50)

,gameweek,dayofweek,date,time,squad_a,score,squad_b,attendance,venue,referee,match_report,notes
0,1,Fri,2021-08-13,20:00,Brentford,2–0,Arsenal,"16,479",Brentford Community Stadium,Michael Oliver,https://fbref.com/en/matches/3adf2aa7/Brentfor...,0
1,1,Sat,2021-08-14,12:30,Manchester Utd,5–1,Leeds United,"72,732",Old Trafford,Paul Tierney,https://fbref.com/en/matches/e62685d4/Manchest...,0
2,1,Sat,2021-08-14,15:00,Leicester City,1–0,Wolves,"31,983",King Power Stadium,Craig Pawson,https://fbref.com/en/matches/0b346a62/Leiceste...,0
3,1,Sat,2021-08-14,15:00,Burnley,1–2,Brighton,"16,910",Turf Moor,David Coote,https://fbref.com/en/matches/4eb36e37/Burnley-...,0
4,1,Sat,2021-08-14,15:00,Watford,3–2,Aston Villa,"20,051",Vicarage Road Stadium,Mike Dean,https://fbref.com/en/matches/814b563c/Watford-...,0
5,1,Sat,2021-08-14,15:00,Chelsea,3–0,Crystal Palace,"38,965",Stamford Bridge,Jonathan Moss,https://fbref.com/en/matches/6f454493/Chelsea-...,0
6,1,Sat,2021-08-14,15:00,Everton,3–1,Southampton,"38,487",Goodison Park,Andy Madley,https://fbref.com/en/matches/c99ebbf5/Everton-...,0
7,1,Sat,2021-08-14,17:30,Norwich City,0–3,Liverpool,"27,023",Carrow Road,Andre Marriner,https://fbref.com/en/matches/c52500ad/Norwich-...,0
8,1,Sun,2021-08-15,14:00,Newcastle Utd,2–4,West Ham,"50,673",St. James' Park,Martin Atkinson,https://fbref.com/en/matches/41091264/Newcastl...,0
9,1,Sun,2021-08-15,16:30,Tottenham,1–0,Manchester City,"58,262",Tottenham Hotspur Stadium,Anthony Taylor,https://fbref.com/en/matches/ff51efc7/Tottenha...,0


In [10]:
pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13159 entries, 0 to 13158
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   gameweek      13159 non-null  object
 1   dayofweek     13159 non-null  object
 2   date          13159 non-null  object
 3   time          13159 non-null  object
 4   squad_a       13159 non-null  object
 5   score         13159 non-null  object
 6   squad_b       13159 non-null  object
 7   attendance    13159 non-null  object
 8   venue         13159 non-null  object
 9   referee       13159 non-null  object
 10  match_report  13159 non-null  object
 11  notes         13159 non-null  object
dtypes: object(12)
memory usage: 1.2+ MB


In [11]:
# Getting rid of empty rows
x = -1
indexes = []
for val in pl['squad_a'].values:
    x += 1
    if val == 0:
        indexes.append(x)
# Check
pl.iloc[indexes].sum()

gameweek        0.0
dayofweek       0.0
date            0.0
time            0.0
squad_a         0.0
score           0.0
squad_b         0.0
attendance      0.0
venue           0.0
referee         0.0
match_report    0.0
notes           0.0
dtype: float64

In [12]:
pl.drop(indexes, axis=0, inplace=True)

In [13]:
pl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11646 entries, 0 to 13158
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   gameweek      11646 non-null  object
 1   dayofweek     11646 non-null  object
 2   date          11646 non-null  object
 3   time          11646 non-null  object
 4   squad_a       11646 non-null  object
 5   score         11646 non-null  object
 6   squad_b       11646 non-null  object
 7   attendance    11646 non-null  object
 8   venue         11646 non-null  object
 9   referee       11646 non-null  object
 10  match_report  11646 non-null  object
 11  notes         11646 non-null  object
dtypes: object(12)
memory usage: 1.2+ MB


## Saving Scrapped Data 

In [14]:
pl.to_csv('pl_scraped_data/PL_AllMatches_Scores_Links.csv', index=False)